In [1]:
#| default_exp routes.card

In [2]:
# | exporti
from typing import Optional

import io
import pandas as pd

import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda


In [3]:
#| export
async def get_kpi_definition(auth: dmda.DomoAuth, card_id: str, debug_api: bool = False) -> rgd.ResponseGetData:
    
    url = f"https://{auth.domo_instance}.domo.com/api/content/v3/cards/kpi/definition"

    body = {"urn": card_id}

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='PUT',
        body=body,
        debug_api=False
    )

    return res

#### sample implementation of get_card_kpi

In [4]:
import os 

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

card_id = 1766265020
await get_kpi_definition(card_id = card_id, auth = token_auth)

ResponseGetData(status=200, response={'id': 1766265020, 'urn': '1766265020', 'definition': {'subscriptions': {'big_number': {'name': 'big_number', 'columns': [{'column': 'Amount', 'aggregation': 'SUM', 'alias': 'Sum of Amount', 'format': {'type': 'abbreviated', 'format': '#'}}], 'filters': [], 'orderBy': [], 'groupBy': [], 'fiscal': False, 'projection': False, 'distinct': False, 'limit': 1}, 'main': {'name': 'main', 'columns': [{'column': 'Date', 'calendar': True, 'mapping': 'ITEM'}, {'column': 'Amount', 'aggregation': 'SUM', 'mapping': 'VALUE'}, {'column': 'Vendor', 'mapping': 'SERIES'}], 'filters': [], 'dateGrain': {'column': 'Date', 'dateTimeElement': 'DAY'}, 'orderBy': [], 'groupBy': [{'column': 'Vendor'}, {'column': 'Date', 'calendar': True}], 'fiscal': False, 'projection': False, 'distinct': False}}, 'formulas': [], 'conditionalFormats': [], 'annotations': [], 'slicers': [], 'title': 'Vendor Amounts Dates', 'description': '', 'chartVersion': '9', 'charts': {'main': {'component': 

In [5]:
#| export
async def get_card_metadata(auth: dmda.DomoAuth, card_id: str, debug_api: bool = False) -> rgd.ResponseGetData:
    optional_params = "metadata,certification,datasources,owners,problems"
    url = f"https://{auth.domo_instance}.domo.com/api/content/v1/cards?urns={card_id}&parts={optional_params}"

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='GET',
        debug_api=debug_api
    )

    return res

#### sample implementation of get_card_metadata

In [6]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

card_id = 1766265020
await get_card_metadata(card_id=card_id, auth=token_auth)


ResponseGetData(status=200, response=[{'metadata': {'chartType': 'badge_vert_stackedbar', 'chartVersion': '7', 'currentLabel': 'Sum of Amount', 'historyId': 'b9e50cd4-19c8-44f2-a44c-6c72bd432125', 'allTime': '{"groupName":"Other","name":"DEFAULT","type":"Other","selected":true,"estimatedSeconds":-1,"dateLabel":"","column":"Date"}', 'calendar': 'default', 'columnAliases': '{}', 'columnFormats': '{}', 'defaultDateGrain': 'Day', 'SummaryNumberFormat': '{"type":"number","format":"#A"}'}, 'owners': [{'id': '1893952720', 'type': 'USER', 'displayName': 'Jae Wilson1'}], 'datasources': [{'dataSourceId': 'fd70d83b-38dc-40a2-9349-357ec4321d3e', 'dataSourceName': 'Vendor Amounts Dates', 'displayType': 'webform', 'dataType': 'webform', 'providerType': 'webform', 'isSampleData': False, 'lastUpdated': 1603767029732, 'adc': False, 'phase': None, 'state': 'SUCCESS'}], 'certification': {'state': 'NOT_CERTIFIED', 'adminCertified': False}, 'urn': '1766265020', 'id': 1766265020, 'type': 'kpi', 'created': 1

In [7]:
#| export
def generate_body_search_cards_admin_summary(page_ids: [str] = None,
                                             searchPages: bool = True,
                                             cardSearchText: str = '',
                                             pageSearchText: str = '') -> dict:
    body = {
        "ascending": True,
        "orderBy": "cardTitle"
    }

    

    if cardSearchText:
        body.update({'cardTitleSearchText': cardSearchText, "includeCardTitleClause": True })

    if pageSearchText:
        body.update({'pageTitleSearchText': pageSearchText, "includePageTitleClause": True, "notOnPage": False})

    if page_ids:
        body.update({'pageIds': page_ids})

    return body

In [8]:
# | export
async def search_cards_admin_summary(auth: dmda.DomoAuth,
                                     body: dict,

                                     maximum :int = None,
                                     
                                     debug_api: bool = False,
                                     debug_loop: bool = False,
                                     session : httpx.AsyncClient  = None,
                                     wait_sleep: int = 3

                                     ) -> rgd.ResponseGetData:

    limit = 100
    offset = 0
    loop_until_end = False if maximum else True

    url = f'https://{auth.domo_instance}.domo.com/api/content/v2/cards/adminsummary?skip={offset}&limit={limit}'
    
    offset_params = {
        "offset": "skip",
        "limit": "limit",
    }

    def arr_fn(res):
        return res.response.get('cardAdminSummaries')

    res = await gd.looper(
        auth=auth,
        method="POST",
        url=url,
        arr_fn=arr_fn,
        offset_params=offset_params,
        limit=limit,
        skip=offset,
        body = body,
        maximum = maximum,
        session=session,
        debug_api=debug_api,
        debug_loop=debug_loop,
        loop_until_end=loop_until_end,
        wait_sleep=wait_sleep
    )
    return res


#### sample implementation of search_cards_admin_summary

In [9]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

body = generate_body_search_cards_admin_summary()

print(body)

card_id = 1766265020
res = await search_cards_admin_summary(body = body, maximum = 5,  auth=token_auth, debug_api = False, debug_loop= False, wait_sleep = 5)

res.response[0:5]


{'ascending': True, 'orderBy': 'cardTitle'}


[{'owners': [{'id': '722128535',
    'type': 'USER',
    'displayName': 'Miguel Guedes'}],
  'id': 332819959,
  'type': 'kpi',
  'badgeUpdated': 1632747565000,
  'title': ' Forecast-Sales_MG',
  'locked': False,
  'pageHierarchy': [{'pageId': -100000, 'title': 'Overview'}]},
 {'owners': [{'id': '1251168784',
    'type': 'USER',
    'displayName': 'Andre Wieczorkiewicz'}],
  'id': 231472916,
  'type': 'kpi',
  'badgeUpdated': 1620397890000,
  'title': '# Active Users',
  'locked': False,
  'pageHierarchy': [{'pageId': 155942344, 'title': 'Copy of Data Stories'}]},
 {'owners': [{'id': '1251168784',
    'type': 'USER',
    'displayName': 'Andre Wieczorkiewicz'}],
  'id': 1025952623,
  'type': 'kpi',
  'badgeUpdated': 1620630492000,
  'title': '# Active Users',
  'locked': False,
  'pageHierarchy': [{'pageId': 155942344, 'title': 'Copy of Data Stories'}]},
 {'owners': [{'id': '1251168784',
    'type': 'USER',
    'displayName': 'Andre Wieczorkiewicz'}],
  'id': 391084879,
  'type': 'kpi',


In [10]:
#| hide
import nbdev
nbdev.nbdev_export()


Bad pipe message: %s [b'\xbdG\x15K;\x08{\xf8\xd9\x95\xaf\xa6\xd2\xf1.\xe0\xe1, ;)\xc1{\xfb8\xbdg9,Y']
Bad pipe message: %s [b'\xa7\x1a!\xc7A\xe3\xac\xfe\x80\x1dA\xdb\x92\xf4\x8cT\x17\xb1 K>\xff\xa8\x05j~\xa2Lw\xbb\x80\xb6FZ\x17\xa2\xa8\xa1\xfe\xdf\x889p\xf2\xfcb\xfc\xbc`\xae\x84\x00\x08\x13\x02\x13\x03\x13', b'']
Bad pipe message: %s [b"\xc9\xa4\xb9\x07\xee5<\xd0\x06\x85'\xb9h\x0c\x9c<S\x1b\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0"]
Bad pipe message: %s [b"R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t1"]
Bad pipe message: %s [b'.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18